In [1]:
#基于urllib，采用Apache2 Licensed开源协议的HTTP库
import requests
#美丽汤，网页解析库，可代替正则表达式
from bs4 import BeautifulSoup
#sys模块包含了与python解释器和它环境有关的函数
import sys
#正则表达式，检查一个字符串是否与某种模式匹配
import re

In [2]:
#创建一个类，就像requests.get()一样，父子节点，包含关系
class downloader(object):
    #初始化
    def __init__(self):
        #获取小说网址
        #比如http://www.bxwx4.com/
        print("Please input the url of novel website")
        self.server = input('> ')
        #获取需要下载的小说url
        #比如http://www.bxwx4.com/104039_41/
        print("Please input the url of book's list")
        self.target = input('> ')
        #模拟浏览器登录，我从电脑上的世界之窗复制的
        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134'}
        #存放章节名
        self.names = []
        #存放章节链接
        self.urls = []
        #章节数
        self.nums = 0
        
    #获取小说名字，在创建txt文档时使用
    def get_bookname(self):
        #发送get请求，得到url源码
        req = requests.get(url = self.target,headers = self.headers)
        #.text转化成文本？
        html = req.text
        #这里使用的正则，因为现在的我无法用美丽汤来获取。。
        bookname = re.findall(r'<h1>(.*?)</h1>',html)[0]
        return bookname
    
    #得到需要下载的链接
    def get_download_url(self):
        req = requests.get(url = self.target,headers = self.headers)
        html = req.text
        #用美丽汤来解析，要添加html.parser来解析，不然会报错，至少在我这是
        soup = BeautifulSoup(html,'html.parser')
        #根据html源码查看自己所需要的部分，这里就是find章节目录
        div = soup.find_all('div',id='list')
        #这个[0],现在也不太清楚，似乎是数组的第一个，也不太清楚html中的存放
        lists = BeautifulSoup(str(div[0]),'html.parser')
        #似乎每个网站存放list都是<a href="".../a>
        #找出所有章节
        lists_a = lists.find_all('a')
        self.nums = len(lists_a)
        for each in lists_a:
            #添加章节名进数组
            self.names.append(each.string)
            #添加每一章节对应的url
            self.urls.append(self.server + each.get('href'))
            
    #获取每一章的内容        
    def get_contents(self,target):
        req = requests.get(url = target)
        html = req.text
        soup = BeautifulSoup(html,'html.parser')
        texts = soup.find_all('div',id = 'content')
        #源码中会有很多<br />或者其他符号，用\n替换
        #有些网站会隐藏正文内容，爬取不到试了网友说的使用
        #texts[0].text.replace('','\n').replace('br /','\n')
        #正文是显示出来了不过是一个字一行，也不知道怎么解决
        texts = texts[0].text.replace('br /','\n')
        return texts
    
    #将内容写入文件中
    def writer(self,name,path,text):
        write_flag = True
        #打开路径，没有就创建，以utf-8的编码方式
        with open(path,'a',encoding='utf-8') as f:
            f.write(name +'\n')
            #对章节内容按一行一行输入
            f.writelines(text)
            f.write('\n\n')
            
            
#主函数
if __name__ == "__main__":
    dl = downloader()
    #父子，包含关系
    #因为初始化的时候给了目标网址，self已经在downloader这个类中共享了
    #这个函数使用后已经获取到完整的章节名数组，以及对应的url数组
    dl.get_download_url()
    bookname = dl.get_bookname()
    for i in range(dl.nums):
        #self已经在类中用了，只需输入后面三个参数
        #一章节为单位进行写入
        #循环一次写完一章节
        #以%后的bookname传入%s
        dl.writer(dl.names[i],'%s.txt' % bookname,dl.get_contents(dl.urls[i]))
        #显示下载进度，以百分比形式，精度为小数点后三位
        #float(i*100/dl.nums)传入%.3f,  %%输出为%
        # \r 默认表示将输出的内容返回到第一个指针，这样的话，后面的内容会覆盖前面的内容
        sys.stdout.write("已下载%.3f%%" % float(i*100/dl.nums) + '\r')
        #stdout有缓冲区，该函数用于刷新stdout
        sys.stdout.flush()
    print('%s下载完成' % bookname)
    
    

Please input the url of novel website
> http://www.bxwx4.com/
Please input the url of book's list
> http://www.bxwx4.com/146521_43/
悲伤逆流成河下载完成


In [ ]:
#一些问题
#一：用jupyter运行时进度不能显示，把code保存成.py文件用powershell可以显示
#二：很多时候，在下载中途会与服务器断开连接导致下载失败，只能下载部分。有时候过段时间就行，就像重启大法一样
#三：该code过于简单，就像是在理想情况下做仿真一样，没有触及放爬虫机制
#四：下载速度过慢，没有使用多线程
#五：代理ip
#六：还可以尝试一下从第一章开始下载，根据下一章链接来下载全文，不再章节目录获取所有url
#七：为了代码美观，就下载了悲伤逆流成河这篇不大的小说，因为测试了基本玄幻小说都死在了三次无法链接上，就有错误报出来
#最后，这个代码是根据https://blog.csdn.net/c406495762/article/details/78123502来编写的